In [1]:
import os
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
PATH = r"C:\Users\UmudovBM\Desktop\Валидация Рули\Новая папка" # путь к отчетам сессиям
sep = "\\"

In [3]:
correct_answer = {'Оборудование «Рули»': 'Все перечисленные варианты',
                  'Как происходит поиск арендаторов?': 'Проверенные пользователи сервиса «Рули» отправляют заявки владельцам автомобилей для подтверждения',
                  'Как застраховать аренду в «Рули»?': 'Подключить опцию «КАСКО» в приложении и страховку оплатит арендатор',
                  'Почему «Рули» – это выгодно?': 'Сервис позволяет получать доход от сдачи автомобиля в аренду и снизить тем самым расходы на его содержание',
                  'Почему «Рули» – это безопасно?': 'все перечисленные варианты',
                  'Почему «Рули» – это удобно?': 'все перечисленные варианты',
                  '«Рули» – это': 'Новый городской сервис для автовладельцев',
                  'А как насчет угона автомобиля?': 'Все перечисленные варианты',
                  'Что получает автовладелец при подключении к сервису?': 'перегон автомобиля в сервис, бесплатное оборудование и его установку, а также ОСАГО на три месяца и бесплатный доступ к сервису',
                  'Зачем это нужно городу?': 'Чем больше людей пользуется одним автомобилем, тем больше разгружается дорожная сеть города. Это положительно влияет на облик города и экологию',
                  'Что происходит после подключения автомобиля к сервису «Рули»?': 'владелец сам решает, когда и кому сдать автомобиль в аренду'}

In [4]:
ready_answer = 'Готовы ли вы подключить свой автомобиль?'
answer_not = 'Ваш комментарий'
ready_session = {}

In [5]:
win_poll_session = []
win_poll ={}
loser_session = {}
for id_task in os.listdir(PATH):
    for id_session in os.listdir('{0}{1}{2}'.format(PATH, sep, id_task)):
        answer_sum = 0
        file_name = os.listdir('{0}{1}{2}{1}{3}.'.format(PATH, sep, id_task, id_session))
        df = pd.read_excel('{0}{1}{2}{1}{3}{1}{4}'.format(PATH, sep, id_task,id_session,*file_name))
        for question,answer in correct_answer.items():
            if df[df[df.columns[0]] == question].iat[0,1].lower() == answer.lower():
                answer_sum += 1
        if answer_sum > 7 :
            win_poll_session.append(int(id_session))
            win_poll[id_session] = answer_sum
        else:
            loser_session[id_session] = answer_sum

        ready_session[int(id_session)]=  ','.join(str(i) for i in [df[df[df.columns[0]] == ready_answer].iat[0,1],
                                                                   df[df[df.columns[0]] == answer_not].iat[0,1]])

In [6]:
df_ready_session = pd.DataFrame(list(ready_session.items()), columns=['id_session', 'ready_answer'])

In [7]:
df_loser_session_count = pd.DataFrame(list(loser_session.items()), columns=['id_session', 'answer_loser'])
df_loser_session_count.answer_loser.value_counts()

7    1
Name: answer_loser, dtype: int64

In [8]:
win_poll_count = pd.DataFrame(list(win_poll.items()), columns=['id_session', 'answer_win'])
win_poll_count.answer_win.value_counts()

11    3
9     1
Name: answer_win, dtype: int64

In [9]:
# Выгрузка из Базы Данных по сессиям и юзерам
PATH_database = r"C:\Users\UmudovBM\Desktop\Валидация Рули\Выгрузки BI"

In [10]:
unload_database = ['ГЗ_2_03 Выгрузка sessions.xlsx','ГЗ_2_04 Выгрузка user.xlsx']

In [11]:
df_users = pd.read_excel('{0}{1}{2}'.format(PATH_database, sep,unload_database[1]))

In [12]:
df_sessions = pd.read_excel('{0}{1}{2}'.format(PATH_database, sep,unload_database[0]))

In [13]:
def rename_columns(df):
    columns = [str(i).replace(' ', '_') for i in df.iloc[1]]
    eld_columns = [i for i in df.columns]
    dict_columns = {}
    for i in range(len(columns)):
        dict_columns[eld_columns[i]] = columns[i]
    df.rename(columns=dict_columns,inplace=True)
    df.drop([0,1,2], inplace=True)

In [14]:
rename_columns(df_users)
rename_columns(df_sessions)

In [15]:
df_users.drop(df_users.columns.difference(['id','firstname', 'lastname', 'middlename','email','phone', ]), 1, inplace=True)

In [16]:
import datetime
date = pd.Timestamp('2023-02-08 22:52:59') # Ограничение по дате (наша выгрузка)
date

Timestamp('2023-02-08 22:52:59')

In [17]:
df_sessions= df_sessions[(df_sessions.task_id == 18089) &
                         (df_sessions.status == 6) &
                         (df_sessions.updated_at < date)] # сессии рули ожидающие подтверждения
# df_sessions = df_sessions[(df_sessions['task_id'] == 18089) &
                         # (df_sessions['status'] == 6) &
                         # (df_sessions['updated_at'] < date)] 

In [18]:
df_sessions.drop(df_sessions.columns.difference(['id','updated_at', 'task_id', 'status', 'user_id']), 1, inplace=True)

In [19]:
df_sessions_users = df_sessions.merge(df_users, how='inner',left_on= 'user_id', right_on= 'id' ).drop(columns='id_y')

In [20]:
df_ready_session = df_sessions_users.merge(df_ready_session,how='inner',left_on= 'id_x', right_on= 'id_session').\
    drop(columns='id_session')

In [21]:
def re_search(x):
    line = re.search(r'[\w\W]*@', str(x))
    if line:
        return line.group()

In [22]:
df_ready_session['pattern'] =  df_ready_session['email'].apply(lambda x: re_search(x))

In [23]:
# выгрузка Заказчика
df_unload_customer = pd.read_csv(r"C:\Users\UmudovBM\Desktop\Валидация Рули\Отчеты заказчика\11483_28148_01_12_2022_13_02_2023_cases_540930123f6dfb4e7a968976e4aa035a.csv", sep=';', on_bad_lines='skip')

In [24]:
def search_unload_customer(x):
    contol_string = ','.join(list(df_unload_customer['Отправитель'].values))
    find_email = re.search(str(x), contol_string)
    if find_email:
        return True
    return False

In [25]:
df_ready_session['win_email'] = df_ready_session['pattern'].apply(lambda x: search_unload_customer(x))

In [26]:
win_win = df_ready_session[df_ready_session.id_x.isin(win_poll_session) & df_ready_session.win_email == True]

In [27]:
win_win['result'] = ['taskCompleted' for i in range(win_win.shape[0])]

In [28]:
loser_game = df_ready_session[~df_ready_session.id_x.isin(win_win.id_x)]

In [29]:
loser_game['result'] = ['taskNotCompleted' for i in range(loser_game.shape[0])]

In [30]:
loser_game.shape[0],win_win.shape[0]

(1, 3)

In [31]:
session_update = win_win[['id_x', 'result']].append(loser_game[['id_x', 'result']])

In [32]:
users_answers = win_win[['email', 'firstname', 'lastname', 'middlename', 'ready_answer']]

In [33]:
users_answers['ФИО'] = users_answers['lastname'] + ' ' + users_answers['firstname'] + ' ' + users_answers['middlename']

In [34]:
def set_answer_1(data):
    if data[:3] == 'Нет':
        return data[4:]

In [35]:
users_answers['comment'] = users_answers['ready_answer'].apply(set_answer_1)

In [36]:
def set_answer_2(data):
    if data[:3] == 'Нет':
        return data[:3]
    if data[:2] == 'Да':
        return data[:2] 

In [37]:
users_answers['ready_answer '] = users_answers['ready_answer'].apply(set_answer_2)

In [38]:
users_answers_yes = users_answers[users_answers['ready_answer '] == 'Да']

In [39]:
users_answers_yes = users_answers_yes[['ФИО', 'email', 'ready_answer ']]

In [40]:
users_answers_no = users_answers[users_answers['ready_answer '] == 'Нет']

In [41]:
users_answers_no = users_answers_no[['ФИО', 'email', 'ready_answer ', 'comment']]

In [42]:
with pd.ExcelWriter(r"C:\Users\UmudovBM\Desktop\Валидация Рули\Для заказчика\Ответы пользователей 09.02.xlsx".format(sep)) as writer:
    users_answers_yes.to_excel(writer, sheet_name="Да", index=False)
    users_answers_no.to_excel(writer, sheet_name="Нет", index=False)

In [43]:
session_update.to_excel(r"C:\Users\UmudovBM\Desktop\Валидация Рули\Обновление сессий\Обновление сессий 09.02.xlsx", index=False, header=None)

In [44]:
#with pd.ExcelWriter(r'C:\Users\UmudovBM\Desktop\Валидация Рули\Результаты\result_wheels 25.01.xlsx'.format(sep)) as writer:
    #win_win.to_excel(writer, sheet_name="Победители")
    #loser_game.to_excel(writer, sheet_name="Неудачники")
    #df_ready_session.to_excel(writer, sheet_name = "Ответы" )